In [3]:
%reload_ext autoreload
%autoreload 2

import voxelmorph as vxm
import numpy as np
import tensorflow as tf
import os
import subtle.utils.io as suio
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
plt.set_cmap('gray')
plt.rcParams['figure.figsize'] = (10, 8)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

<Figure size 720x576 with 0 Axes>

In [1]:
vxm_affine = vxm.networks.VxmAffine(inshape=(128, 256, 256), batch_norm=False, constraint_params=False,
                                          enc_only=True)

NameError: name 'vxm' is not defined

In [ ]:
vxm_affine.model.load_weights('/home/srivathsa/projects/studies/gad/vmorph/checkpoints/affine/mse/0010.h5')

In [ ]:
img_list = '/home/srivathsa/projects/studies/gad/vmorph/val_affine_sc.txt'
generator = vxm.generators.SubtleGadLoader(img_list, batch_size=1, shuffle=False, affine=True, constraint_params=False)

In [ ]:
(moving, fixed) = generator.__getitem__(1)
moving = moving[0]
fixed = fixed[0]
pred = vxm_affine.model.predict([moving, fixed])

In [ ]:
tmp_model = Model(inputs=[vxm_affine.model.layers[0].input, vxm_affine.model.layers[1].input], 
                  outputs=vxm_affine.model.layers[-2].output)
aff_mtrx = tmp_model.predict([moving, fixed])
print(aff_mtrx)

In [ ]:
plt.imshow(pred[0, 98, ..., 0])
print(aff_mtrx)
print(pred.min(), pred.max())

In [ ]:
p1 = np.load('1.npy')
p2 = np.load('2.npy')

plt.imshow(p1-p2)
plt.colorbar()

In [ ]:
loss_fn = vxm.losses.MSE(1.0).loss
vxm_affine.model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss=loss_fn)

In [ ]:
vxm_affine.model.fit_generator(generator, epochs=2, verbose=1)

In [ ]:
model = vxm.networks.Transform(inshape=(196, 240, 240), affine=True)

In [ ]:
data = suio.load_file('/home/srivathsa/projects/studies/gad/tiantan/preprocess/data/NO13.h5', 
                        params={'h5_key': 'all'})

ip_img = data[1, :, 0][None, ..., None]

In [ ]:
# trf_params = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])[None]
trf_params = np.array([
    [0.99818, 0.0201, -0.0012, -0.197], 
    [-0.001751, 0.999811, 0.0025, -0.617], 
    [0.0124535, -0.003, 0.997072, 0.095]]
)[None]

In [ ]:
ip_trf = model.predict([ip_img, trf_params])[0, ..., 0]

In [ ]:
ip_sl = ip_img[0, 98, ..., 0]
plt.imshow(ip_trf[98])

In [ ]:
for fpath in sorted(glob('/home/srivathsa/projects/studies/gad/vmorph/affine_data/*.npz')):
    print(fpath.replace('affine_data', 'affine_data_sc'))
#     data = np.load(fpath)['data']
#     print(fpath.split('/')[-1], data.min(), data.max())

In [ ]:
plt.imshow(data[1, 98] - data[0, 98])

In [ ]:
fc = open('/home/srivathsa/projects/studies/gad/vmorph/train_affine.txt', 'r').read()
fc = [f.replace('affine_data', 'affine_data/sc') for f in fc.split('\n') if f != '']
print('\n'.join(fc))

In [ ]:
val_data = np.load('/home/srivathsa/projects/studies/gad/vmorph/affine_data/NO31.npz')['data']

plt.imshow(val_data[0, 98])